In [1]:
import ipywidgets as widgets
from IPython.display import display
from plot_functions import *
import requests

DriverError: ../data/suburb-2-vic.geojson: No such file or directory

## Weather Data (BoM)

In [ ]:
BOM_API_BASE_URL = "http://localhost:9090/api"

def get_suburbs():
    response = requests.get(f"{BOM_API_BASE_URL}/suburbs")
    return response.json() if response.status_code == 200 else {}

def get_crime_weather_data(suburb):
    response = requests.get(f"{BOM_API_BASE_URL}/{suburb}/crime-weather")
    return response.json() if response.status_code == 200 else {}

def get_suburbs_with_weather():
    all_suburbs = []
    all_suburbs_data = get_suburbs()
    for suburb_data in all_suburbs_data:
        all_suburbs.append(suburb_data['suburb_name'])
    suburbs_data_with_weather = []
    for suburb_name in all_suburbs:
        crime_weather_data = get_crime_weather_data(suburb_name)
        try:
            if crime_weather_data['average_temp'] is not None:
                suburbs_data_with_weather.append(crime_weather_data)
        except KeyError:
            pass
    return suburbs_data_with_weather

def get_by_total_offences():
    return sorted(get_suburbs_with_weather(), key=lambda x: x['total_offences'], reverse=True)

## Sentiment Data (Mastodon)

In [ ]:
MASTODON_API_BASE_URL = "http://localhost:9090/api/sentiment"

def get_sentiment_data(year):
    response = requests.get(f"{MASTODON_API_BASE_URL}/{year}")
    return response.json() if response.status_code == 200 else {}

## Button On Click Handlers

In [ ]:
# Button and Button Click Handler
def handle_s1_click(button,plot_output):
    data = get_by_total_offences()[:10]
    suburbs = [entry['suburb_name'] for entry in data]
    with plot_output:
        plot_output.clear_output(wait=True)
        plot_map('crime',suburbs)
        plot_scatter_chart('average_temp',data)

def handle_s2_click(button,plot_output):
    data = get_by_total_offences()[:10]
    suburbs = [entry['suburb_name'] for entry in data]
    with plot_output:
        plot_output.clear_output(wait=True)
        plot_map('crime',suburbs)
        plot_scatter_chart('average_rain',data)

def handle_s3_click(button,plot_output):
    data = get_by_total_offences()[:10]
    with plot_output:
        plot_output.clear_output(wait=True)
        plot_bar_chart('average_temp',data)

def handle_s4_click(button,plot_output):
    data = get_by_total_offences()[:10]
    with plot_output:
        plot_output.clear_output(wait=True)
        plot_bar_chart('average_rain',data)
        
def handle_s5_click(button,plot_output):
    years = [2018,2019]
    data = [get_sentiment_data(year) for year in years]
    with plot_output:
        plot_output.clear_output(wait=True)
        plot_pie_chart(data)

## User Interface

In [ ]:
guide = """
    <b>Welcome to the Victoria Crime Analysis System!</b>
    <br>
    In light of the tragic knife attack at Westfield Bondi Junction in Sydney, there's been a renewed focus across Australia on developing effective crime prevention strategies.
    Both citizens and authorities are actively seeking effective solutions to prevent such alarming events. 
    The <b>Victoria Crime Analysis System</b> is your one-stop shop for data-powered 
    insights into crime trends within Victoria. We leverage big data analytics to uncover factors influencing criminal activity in urban areas.
    <hr>
    🔍 Simply click the search button next to your query, and our system will deliver a real-time response! 
    <hr>
"""
queries = [
    ("<b>Query 1: Do weather patterns influence the number of crimes?</b>", [
        ('<b>1.1.</b> Does temperature influence the number of crimes?', handle_s1_click),
        ('<b>1.2.</b> Does precipitation influence the number of crimes?', handle_s2_click)
    ]),
    ("""
        <b>Query 2: Do weather patterns influence the types of crimes?</b>
        <br>
        Type A crimes typically involve violent offenses against individuals, while Type B crimes encompass property-related and deceptive offenses.
     """, [
        ('<b>2.1.</b> Does temperature affect the types of crimes?', handle_s3_click),
        ('<b>2.2.</b> Does precipitation affect the types of crimes?', handle_s4_click)
    ]),
    ('<b>Query 3: Is public sentiment correlated to the number of crimes?</b>', [
        ('<b>3.1.</b> Is public sentiment correlated to the number of crimes?', handle_s5_click)
    ])
]

# Widgets for display
plot_output = widgets.Output()
guide_html = widgets.HTML(guide)

# Create the rows for each query and sub-query
rows = [guide_html]

for query, sub_queries in queries:
    query_html = widgets.HTML(value=query)
    rows.append(query_html)
    for sub_query, handler in sub_queries:
        sub_query_html = widgets.HTML(value=sub_query)
        button = widgets.Button(description="Search")
        button.on_click(lambda button, h=handler: h(button, plot_output))
        rows.append(widgets.HBox(children=[sub_query_html, button]))

# Display Container
container = widgets.VBox(children=rows + [plot_output])
display(container)